In [2]:
import abc


class Multivector(abc.ABC):
    @abc.abstractmethod
    def __add__(self, other: "Multivector") -> "Multivector":
        ...

    @abc.abstractmethod
    def __neg__(self) -> "Multivector":
        ...

    def __sub__(self, other: "Multivector") -> "Multivector":
        return self + -other

    def __mul__(self, other: "Multivector") -> "Multivector":
        return self.product(other)

    @abc.abstractmethod
    def inverse(self) -> "Multivector":
        ...

    def __truediv__(self, other: "Multivector") -> "Multivector":
        return self * other.inverse()
    
    @abc.abstractmethod
    def product_identity(self) -> 'Multivector':
        ...

    @abc.abstractmethod
    def product(self, other: "Multivector") -> "Multivector":
        ...

    @abc.abstractmethod
    def inner(self, other: "Multivector") -> "Multivector":
        ...

    def exterior(self, other: "Multivector") -> "Multivector":
        return self.product(other) - self.inner(other)
    
    @abc.abstractmethod
    def repr(self) -> list[float]:
        ...

In [51]:
import math
import collections
import itertools


def axis_multiplication(a: str, b: str, basis: list[str]) -> tuple[str, int]:
    """returns the multiplication between axis `a` and `b` as a tuple of `(abs(ab), sign)`"""

    combined = a + b

    sign = 1
    # reduce
    while True:
        # find a duplicate
        counts = collections.defaultdict(lambda: [])
        for i, c in enumerate(combined):
            counts[c].append(i)
            if len(counts[c]) > 1:
                # c is repeated
                repeat = c
                break
        else:
            break

        p1, p2 = counts[repeat]

        # count distance between them
        if (p2 - p1) % 2 == 0:
            sign *= -1

        # remove p1 and p2
        combined = combined[:p1] + combined[p1 + 1 : p2] + combined[p2 + 1 :]

    # find the corresponding basis
    for target in basis:
        if set(target) == set(combined):
            break
    else:
        raise Exception("no target matching " + combined + " found")

    # sort into basis
    for p1, c in enumerate(target):
        p2 = combined.index(c)
        if (p2 - p1) % 2 == 1:
            sign *= -1

        # move c into index p1
        combined = combined[:p2] + combined[p2 + 1 :]
        combined = combined[:p1] + c + combined[p1:]

    return combined, sign


def generate_basis(axis: str = "xyz") -> list[str]:
    """returns a list of basis given the axis names"""
    basis = []
    for i in range(len(axis) + 1):
        for choice in itertools.combinations(axis, i):
            basis.append("".join(choice))

    return basis


def generate_product_table(basis: list[str]) -> tuple[list[list[int]], list[list[int]]]:
    """
    Computes the product and sign tables for the basis list as `(product_table, sign_table)`.

    The `x = sign_table[i][j] * product_table[i][j]` represents the element needed such that
    `basis[i] * x = basis[j]`.
    """
    sign = []
    output = []
    for lhs in basis:
        row_sign = []
        row_output = []
        for target in basis:
            # find x where lhs * x = target
            for x in basis:
                res, s = axis_multiplication(lhs, x, basis)
                if res == target:
                    row_output.append(basis.index(x))
                    row_sign.append(s)
                    break
            else:
                raise Exception("no inverse found for " + lhs + " and " + target)

        sign.append(row_sign)
        output.append(row_output)

    return output, sign

    # define multiplication between axis


# generate_product_table()
# axis_multiplication('zzxy', 'x', generate_basis())
# output, sign = generate_product_table(generate_basis('xyzabc'))
# print(np.array(output))
print(generate_basis("xyzabc"))
# print(axis_multiplication('yz', 'z', generate_basis()))

['', 'x', 'y', 'z', 'a', 'b', 'c', 'xy', 'xz', 'xa', 'xb', 'xc', 'yz', 'ya', 'yb', 'yc', 'za', 'zb', 'zc', 'ab', 'ac', 'bc', 'xyz', 'xya', 'xyb', 'xyc', 'xza', 'xzb', 'xzc', 'xab', 'xac', 'xbc', 'yza', 'yzb', 'yzc', 'yab', 'yac', 'ybc', 'zab', 'zac', 'zbc', 'abc', 'xyza', 'xyzb', 'xyzc', 'xyab', 'xyac', 'xybc', 'xzab', 'xzac', 'xzbc', 'xabc', 'yzab', 'yzac', 'yzbc', 'yabc', 'zabc', 'xyzab', 'xyzac', 'xyzbc', 'xyabc', 'xzabc', 'yzabc', 'xyzabc']
('y', 1)


In [61]:
import numpy as np

MULTIVECTOR3_PRODUCT_TABLE, MULTIVECTOR3_PRODUCT_SIGN = generate_product_table(
    generate_basis("xyz")
)
# MULTIVECTOR3_PRODUCT_TABLE = [
#     [0, 1, 2, 3, 4, 5, 6, 7],
#     [1, 0, 4, 6, 2, 7, 3, 5],
#     [2, 4, 0, 5, 1, 3, 7, 6],
#     [3, 6, 5, 0, 7, 2, 1, 4],
#     [4, 2, 1, 7, 0, 6, 5, 3],
#     [5, 7, 3, 2, 6, 0, 4, 1],
#     [6, 3, 7, 1, 5, 4, 0, 2],
#     [7, 5, 6, 4, 3, 1, 2, 0],
# ]

# MULTIVECTOR3_PRODUCT_SIGN = [
#     [0, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 1, 0, 0, 1, 0],
#     [0, 1, 0, 0, 1, 0, 0, 0],
#     [0, 0, 1, 0, 0, 1, 0, 0],
#     [0, 1, 0, 0, 1, 1, 0, 1],
#     [0, 0, 1, 0, 0, 1, 1, 1],
#     [0, 0, 0, 1, 1, 0, 1, 1],
#     [0, 0, 0, 0, 1, 1, 1, 1],
# ]


class Multivector3(Multivector):
    c: float
    x: float
    y: float
    z: float
    xy: float
    xz: float
    yz: float
    xyz: float

    def __init__(self, c=0, x=0, y=0, z=0, xy=0, xz=0, yz=0, xyz=0):
        self.c = c
        self.x = x
        self.y = y
        self.z = z
        self.xy = xy
        self.xz = xz
        self.yz = yz
        self.xyz = xyz

    def __add__(self, other):
        return Multivector3(
            self.c + other.c,
            self.x + other.x,
            self.y + other.y,
            self.z + other.z,
            self.xy + other.xy,
            self.xz + other.xz,
            self.yz + other.yz,
            self.xyz + other.xyz,
        )

    def __neg__(self):
        return Multivector3(
            -self.c,
            -self.x,
            -self.y,
            -self.z,
            -self.xy,
            -self.xz,
            -self.yz,
            -self.xyz,
        )

    def product_matrix(self):
        arr = self.repr()

        return [
            [
                arr[MULTIVECTOR3_PRODUCT_TABLE[row][col]]
                * MULTIVECTOR3_PRODUCT_SIGN[row][col]
                for col in range(len(arr))
            ]
            for row in range(len(arr))
        ]

    def inverse(self, eps: float = 0.001):
        """returns the left inverse, a^-1, where a^-1 a = 1"""

        # TODO: Prove if this is the same as the right inverse

        # make product matrix
        mat_np = np.array(self.product_matrix())
        if abs(np.linalg.det(mat_np)) < eps:
            return None

        I = self.product_identity().repr()
        output = np.matmul(np.array([I]), np.linalg.inv(mat_np)).tolist()[0]
        return Multivector3(*output)

    def repr(self):
        return [self.c, self.x, self.y, self.z, self.xy, self.xz, self.yz, self.xyz]

    def product_identity(self):
        return Multivector3(1)

    def product(self, other: "Multivector3"):
        inputs = self.repr()
        mat_np = np.array(other.product_matrix())

        output = np.matmul(np.array([inputs]), mat_np).tolist()[0]
        return Multivector3(*output)

    def inner(self, other):
        return Multivector3(
            self.c * other.c
            + self.x * other.x
            + self.y * other.y
            + self.z * other.z
            + self.xy * other.xy
            + self.xz * other.xz
            + self.yz * other.yz
            + self.xyz * other.xyz
        )

    @staticmethod
    def get(basis: str):
        names = ["1s", "x", "y", "z", "xy", "xz", "yz", "xyz"]

        if basis not in names:
            return None

        index = names.index(basis)
        outputs = [0] * len(names)
        outputs[index] = 1
        return Multivector3(*outputs)

    def __str__(self):
        outputs = []
        names = ["1s", "x", "y", "z", "xy", "xz", "yz", "xyz"]
        for i, val in enumerate(self.repr()):
            if abs(val) < 0.001:
                continue

            outputs.append(f"{names[i]}={val:.2f}")

        output = ", ".join(outputs)
        return f"({output})"

    def __repr__(self) -> str:
        return f"Multivector3{str(self)}"


class Vector3(Multivector3):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)

    def inverse(self):
        pass

In [63]:
# Multivector3(xz=1, yz=1, c=1) * Multivector3(z=1)


a = Multivector3(1, 2, 1, 1)
b = Multivector3(0, 2)

print(a * b)
print(a.inverse())
print(a * a.inverse())
print(a.inverse() * a)
print(a.inverse().inverse())

(1s=4.00, x=2.00, xy=-2.00, xz=-2.00)
(1s=-0.20, x=0.40, y=0.20, z=0.20)
(1s=1.00)
(1s=1.00)
(1s=1.00, x=2.00, y=1.00, z=1.00)


In [62]:

def project(point: Multivector3, plane: Multivector3):
    # a = amm^-1 = (a dot m + a ext m) m^-1
    #   = a ext m m^-1 + a dot m m^-1
    #   = projection + rejection

    inv = plane.inverse()
    return (point.exterior(plane)) * inv, (point.inner(plane)) * inv

# TODO: Why the negative?
project(Multivector3(x=1), Multivector3(z=1, x=2, y=1))


(Multivector3(y=1.00, z=1.00), Multivector3(x=2.00))